In [1]:
import getpass
import os
from credentials import *

ModuleNotFoundError: No module named 'credentials'

In [44]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint= Azure_OpenAI_OB_Endpoint ,
    openai_api_version="2024-02-15-preview",
    model_name="gpt-4o",
    openai_api_key= Azure_OpenAI_OB_Key,
    openai_api_type="azure",
    temperature=0,
    deployment_name="gpt-4o-deploy",
    #streaming=True,
)

## 1 - Test

In [59]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. 
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")]
)

In [57]:
from langchain_core.output_parsers import StrOutputParser
chain = prompt | llm | StrOutputParser()

contenu_ancien = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
contenu_nouveau = "Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en opérations de banque et en services de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent aux services d'intermédiaires en opérations de banque et en services de paiement doivent s'assurer que ceux-ci sont immatriculés conformément à l'article L. 519-3-1 . Les établissements de crédit, les sociétés de financement, les établissements de monnaie électronique qui fournissent des services de paiement, les établissements de paiement, les intermédiaires en financement participatif, les entreprises d'assurance dans le cadre de leur activité de prêts et les sociétés de gestion mentionnées au premier alinéa de l'article L. 519-2 qui recourent à des intermédiaires ressortissants d'un Etat membre de l'Union européenne ou d'un Etat partie à l'accord sur l'Espace économique européen autre que la France et exerçant sur le territoire français en régime de libre prestation de services ou de libre établissement s'assurent auprès de l'organisme qui tient le registre prévu au I de l'article L. 512-1 du code des assurances que ceux-ci ont effectué les formalités requises à l'article L. 519-9 du présent code."
llm_output = chain.invoke(
    {
        "code": "Code monétaire et financier",
        "old_version": contenu_ancien,
        "new_version": contenu_nouveau,
        "input": "I love programming.",
    }
)

In [58]:
print(llm_output)

L'analyse des modifications apportées à l'article en question du Code monétaire et financier révèle une suppression notable. 

**Ancienne version :**
L'ancienne version de l'article incluait les "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" parmi les entités qui doivent s'assurer que les intermédiaires en opérations de banque et en services de paiement sont immatriculés conformément à l'article L. 519-3-1. De plus, cette catégorie était également mentionnée dans le contexte des intermédiaires ressortissants d'un État membre de l'Union européenne ou d'un État partie à l'accord sur l'Espace économique européen exerçant en France en régime de libre prestation de services ou de libre établissement.

**Nouvelle version :**
Dans la nouvelle version de l'article, la mention des "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts" a été supprimée. Ain

## 2 - Function

In [64]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def llm_legal_change_com(code, old_version, new_version): # "Code monétaire et financier"
    

    prompt = ChatPromptTemplate.from_messages([("system",
            """Tu es un avocat et analyste juridique très expérimenté, tu as pour but d'analyser et commenter les changements 
            réglementaires de chaque article , tu trouveras ici l'ancienne et la nouvelle version de l'article que l on te donne. 
            Tu adopteras un language juridique précis. Tu n'inventeras rien et tu te baseras uniquement sur les données fournies
            Pour avoir du contexte , sache que ce texte est extrait du {code}"""),
        ("human", 
            "Voici l'ancienne version de l'article a analyser {old_version} et voici la nouvelle version du meme article {new_version}")])
    
    chain = prompt | llm | StrOutputParser()
    
    llm_output = chain.invoke({
        "code": code,
        "old_version": old_version,
        "new_version": new_version})
    
    return print(llm_output)

In [65]:
llm_legal_change_com("Code monétaire et financier", contenu_ancien, contenu_nouveau)

L'analyse comparative des deux versions de l'article révèle une modification notable. La nouvelle version de l'article a supprimé une catégorie spécifique d'acteurs économiques, à savoir les "prestataires de services de financement participatif dans le cadre de leurs activités de facilitation d'octroi de prêts". 

Dans l'ancienne version, ces prestataires étaient explicitement mentionnés à deux reprises : 
1. Dans la liste des entités qui doivent s'assurer que les intermédiaires en opérations de banque et en services de paiement sont immatriculés conformément à l'article L. 519-3-1.
2. Dans la liste des entités qui doivent s'assurer que les intermédiaires ressortissants d'un État membre de l'Union européenne ou d'un État partie à l'accord sur l'Espace économique européen ont effectué les formalités requises à l'article L. 519-9 du présent code.

La suppression de cette catégorie dans la nouvelle version de l'article signifie que les prestataires de services de financement participatif 